In [1]:
from dataloading.dataloader import AVDeepfake1m
from dataloading.metadata_generator import generate_metadata
from torch.utils.data import DataLoader

/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load data.

In [2]:
# Define the path to your local dataset
local_data_path = "data/train/"

# generate metadata
generate_metadata(local_data_path)

Existing metadata file data/train/train_metadata.json has been removed.
Metadata saved to data/train/train_metadata.json


In [3]:
# Initialize the dataset
train_dataset = AVDeepfake1m(
    subset="train",  # Specify the subset (e.g., train, val, test)
    data_root=local_data_path,  # Path to your local dataset
    temporal_size=100,  # Temporal size for video frames
    max_duration=30,  # Maximum duration for labels
    fps=25,  # Frames per second
    video_transform=None,  # Optional video transformation
    audio_transform=None,  # Optional audio transformation
    require_match_scores=False,  # Whether to compute match scores
    return_file_name=False,  # Whether to return file names
    is_plusplus=True  # Set to True if using AV-Deepfake1M++
)

# Wrap the dataset in a DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=1,  # Number of samples per batch
    shuffle=True,  # Shuffle the data
    num_workers=4,  # Number of worker threads for loading data
    drop_last=True  # Drop the last incomplete batch
)

Load 2608 data in train.


In [4]:

# Inspect the Contents of the DataLoader
# This cell demonstrates how to iterate over the `train_loader` and print the contents of the first batch.
# Iterate over the DataLoader

for batch in train_loader:
    video, audio, labels = batch[:3]  # Extract video, audio, and labels
    print(f"Video shape: {video.shape}")
    print(f"Audio shape: {audio.shape}")
    print(f"Labels: {labels}")
    break  # Remove this break to inspect all batches

/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torchvision/io/_video_deprecation_warning.py:5: U

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/ben/Thesis/ForgeryLocalization/dataloading/dataloader.py", line 110, in __getitem__
    video = F.interpolate(video.float().permute(1, 0, 2, 3)[None], size=(self.temporal_size, 96, 96))[0]
  File "/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/